# StackSMART
## Predict waiting time to get answers on Stack Overflow
<img src="img/intro" alt="Drawing" style="width: 700px; height: 500px;"/>

## Data
#### Stack Exchange DataDump: https://archive.org/download/stackexchange
#### Stanford University SNAP Network Datasets: http://snap.stanford.edu/data/sx-stackoverflow.html
## Features
#### Post Characteristics: Language used, length of post, tags
#### User Network Characteristics
#### Others: Time of posting
## Model
#### Combination of Linear Regression and Random Forest

In [ ]:
# Random Forest Estimator
md_range = np.arange(2,22,2) #search max_depth = 2:20
msl_range = np.arange(20,35,5) #search min_samples_leaf = 20,25,30,35
ne_range = np.arange(10,40,5) #search n_estimators = 10,15,20,25,30,35
est = RandomForestRegressor()
rf = model_selection.GridSearchCV(
    est, 
    {'max_depth': md_range,
    'min_samples_leaf': msl_range,
    'n_estimators': ne_range},
    cv = 10,
    n_jobs = 5,
    scoring = 'neg_mean_squared_error'
)

## Prediction Result (Mean Absolute Error: 0.83, R Squared: 0.83)
<img src="img/predRealScatter" alt="Drawing" style="width: 600px; height: 600px;"/>

## Top 40 Words that Increase or Decrease Waiting Time
<img src="img/top40Words" alt="Drawing" style="width: 700px; height: 600px;"/>

## Link to App: http://localhost:8888/